In [244]:
#imports

import pymssql
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter
import tracemalloc
import matplotlib.pyplot as plt

In [116]:
# database info

database = "arctic_analysts_capstone"
user = "arctic_analysts"
password  = "ThisPassw0rd!"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

In [117]:
#reading in tables, will be written better

def get_data():
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f'SELECT * FROM house_prices'

    house_price_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {house_price_df.shape[0]}')

    return house_price_df
house_price_df = get_data()

def get_data():
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f'SELECT * FROM building_permits'

    building_permits_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {building_permits_df.shape[0]}')

    return building_permits_df
building_permits_df = get_data()

def get_data():
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f'SELECT * FROM median_income'

    median_income_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {median_income_df.shape[0]}')

    return median_income_df
median_income_df = get_data()

def get_data():
    conn = pymssql.connect(server, user, password, database)
    cursor = conn.cursor()
    
    query = f'SELECT * FROM mortgage_rates'

    mortgage_rates_df = pd.read_sql(query, conn)
    print(f'\nBefore Data Management: {mortgage_rates_df.shape[0]}')

    return mortgage_rates_df
mortgage_rates_df = get_data()


Before Data Management: 757904

Before Data Management: 220509

Before Data Management: 61430

Before Data Management: 585


In [118]:
# all transformation will be probably re-written either better or in databricks before loading into database

# changing mortgage month to merge 
month_list = {
    'JANUARY':'Jan',
    'FEBRUARY':'Feb',
    'MARCH':'Mar',
    'APRIL ':'Apr',
    'MAY':'May',
    'JUNE':'Jun',
    'JULY':'Jul',
    'AUGUST':'Aug',
    'SEPTEMBER':'Sep',
    'OCTOBER':'Oct',
    'NOVEMBER':'Nov',
    'DECEMBER':'Dec'
}
mortgage_rates_df['Month'] = mortgage_rates_df['Month'].map(month_list)

# casting year, fips, and month variables appropriate data types
df_list = [building_permits_df,house_price_df,median_income_df,mortgage_rates_df]
variable_list = ['Year', 'FIPS']
for df in df_list:
    for variable in variable_list:
        if variable in df.columns:
            df[variable] = df[variable].astype('int64')

df_list = [building_permits_df,house_price_df,mortgage_rates_df]
for df in df_list:
    df['Month'] = df['Month'].astype('string')

In [128]:
# filtering down income to only 'overall' age group
income_to_merge = median_income_df[median_income_df['AgeGroup']=='overall']

# merging the four tables, will discuss diff age groups later
house_permit = pd.merge(building_permits_df,house_price_df, on=['FIPS','Year','Month'],how='inner')
house_permit_mortgate = pd.merge(house_permit,mortgage_rates_df, on=['Year','Month'],how='inner')
final_table = pd.merge(house_permit_mortgate,income_to_merge, on=['FIPS','Year'],how='inner')
final_table = final_table[['FIPS','Year','Month','County','NewUnits','NewBuildings','MedianHousePrice','AverageRate','AveragePoints','MedianIncome']]

In [129]:
# transformation of datatype in final table
final_table['MedianHousePrice'] = pd.to_numeric(final_table['MedianHousePrice'], errors='coerce')

#calculate monthly income
final_table['MonthlyIncome'] = final_table['MedianIncome']/12

#calculate montly mortgage payment
#https://www.educba.com/mortgage-formula/

for row in final_table:
    P = final_table['MedianHousePrice']-(final_table['MedianHousePrice']*.2)
    r = (final_table['AverageRate']/100)
    t = 30
    n = 12
    final_table['MonthlyMortgage'] = P * (((r/n) * pow((1+(r/n)),(n*t))) / (pow((1+r/n),(n*t))-1))

# mortgage to income ratio
final_table['mortgage_income_ratio'] = final_table['MonthlyMortgage']/final_table['MonthlyIncome']

#affordability determination
def affordable_condition(x):
    if x <= .25:
        return 'Yes'
    elif np.isnan(x):
        return 'Missing'
    else:
        return 'No'

final_table['affordable'] = final_table['mortgage_income_ratio'].apply(affordable_condition)

In [187]:
# general count of county affordability

final_table['affordable'].value_counts()

In [194]:
final_table

,FIPS,Year,Month,County,NewUnits,NewBuildings,MedianHousePrice,AverageRate,AveragePoints,MedianIncome,MonthlyIncome,MonthlyMortgage,mortgage_income_ratio,affordable
0,1073,2005,Jan,"Jefferson County, Alabama",269,269,128142.0,5.71,0.7,42013,3501.083333,595.639169,0.170130,Yes
1,1073,2005,Feb,"Jefferson County, Alabama",259,259,128828.0,5.63,0.7,42013,3501.083333,593.610695,0.169551,Yes
2,1073,2005,Mar,"Jefferson County, Alabama",278,278,129598.0,5.93,0.7,42013,3501.083333,616.946114,0.176216,Yes
3,1073,2005,May,"Jefferson County, Alabama",302,241,131284.0,5.72,0.6,42013,3501.083333,610.910103,0.174492,Yes
4,1073,2005,Jun,"Jefferson County, Alabama",296,296,131881.0,5.58,0.6,42013,3501.083333,604.350968,0.172618,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66907,55131,2019,Aug,"Washington County, Wisconsin",36,32,266560.0,3.62,0.5,80513,6709.416667,971.920254,0.144859,Yes
66908,55131,2019,Sep,"Washington County, Wisconsin",48,36,267097.0,3.61,0.5,80513,6709.416667,972.676362,0.144972,Yes
66909,55131,2019,Oct,"Washington County, Wisconsin",39,38,267652.0,3.69,0.6,80513,6709.416667,984.354604,0.146712,Yes
66910,55131,2019,Nov,"Washington County, Wisconsin",34,34,268213.0,3.70,0.6,80513,6709.416667,987.631041,0.147201,Yes


In [266]:
# DIFFERENT PATH - rough decision, will be changed

temp_table = final_table.dropna()
temp_table['affordable'].value_counts()

Yes    51814
No     12140
Name: affordable, dtype: int64

In [248]:
final_table.shape

(66912, 14)

In [267]:
# classification model
# X = final_table.iloc[:,[1,4,5,6,7,8,9,10]].values
X = temp_table.drop(['FIPS','Month','County','affordable'],axis = 1).values
y = temp_table['affordable'].values

In [271]:
#Splitting the data into Training Set and Test Set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=0)

#Normalizing the features
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

#Fitting Logistic Regression to Training Set
from sklearn.linear_model import LogisticRegression
classifierObj = LogisticRegression(random_state=0)
classifierObj.fit(X_train, y_train)

#Making predictions on the Test Set
y_pred = classifierObj.predict(X_test)

#Model Accuracy
print(classifierObj.score(X_test,y_test))

#Evaluating the predictions using a Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)



0.99906185502533
[[ 2997     8]
 [    7 12977]]


In [276]:
classifierObj.get_params()


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [280]:
classifierObj.coef_

array([[  0.12806335,   0.12000355,  -0.14689774,  -6.09859637,
         -1.07904862,  -0.07318714,   4.36004133,   4.36004133,
        -11.89998693, -18.42073074]])